In [223]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import RobustScaler, OneHotEncoder

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from pandas.api.types import is_string_dtype

#a faire (FAIT): -tester si dans la partie Robust/onehot on peut pas régler le soucis du nb de columns différent si on fait les 2 en meme temps
#rquared buged des fois

In [224]:
path = 'https://simplonline-v3-prod.s3.eu-west-3.amazonaws.com/media/file/csv/bdfc59ed-c3c7-48ac-a3d3-9e54663f6c1d.csv'
target = 'SalePrice'
initial_df = pd.read_csv('houses.csv')
df = initial_df

df_test = pd.read_csv('test.csv')

#PRE PROC PERSONALISE DB HOUSES
#On supprime toutes les lignes de la DB quand la SaleCondition n'est pas 'Normal'
#df = df[df['SaleCondition']=='Normal'] 

df = df.drop(columns=["Id"])


#on choisis à partir de cb de % de valeures nulles on del les colomnes, si on garde la col, on suppr quand même les lignes nulles
pourcent_null_del_col = 0.15

#df.isnull().sum().head(50)
#df.info()

In [225]:
df.shape, df_test.shape

((1460, 80), (1459, 80))

In [226]:
####### test pre pro #######


list_col_del = []

X = df_test
list_columns = list(X.columns)


for column in list_columns:
    nb_isnull = df_test.isnull().sum()[column]
    if nb_isnull > 0 and is_string_dtype(df_test[column]) is True:
        print(f'Del Col : {column} > trop de isnull ({nb_isnull}/{len(df_test)}) | categorial feature : {is_string_dtype(df_test[column])}')
        df_test = df_test.drop(column, 1)
        list_col_del.append(column)

    elif nb_isnull > ((len(df)*pourcent_null_del_col)) and is_string_dtype(df_test[column]) is False:
        
        print(f'Del Col : {column} > trop de isnull ({nb_isnull}/{len(df_test)}) | categorial feature : {is_string_dtype(df_test[column])}')
        df_test = df_test.drop(column, 1)
        list_col_del.append(column)
        
    elif nb_isnull > 0 and is_string_dtype(df_test[column]) is False:
        df_test[column].fillna((df_test[column].mean()), inplace=True)
        print(f'Replace mean : {column} > trop de isnull ({nb_isnull}/{len(df_test)}) | categorial feature : {is_string_dtype(df_test[column])}')

    
print("\n liste des colomnes à supprimer dans le df train :",list_col_del)


Del Col : MSZoning > trop de isnull (4/1459) | categorial feature : True
Del Col : LotFrontage > trop de isnull (227/1459) | categorial feature : False
Del Col : Alley > trop de isnull (1352/1459) | categorial feature : True
Del Col : Utilities > trop de isnull (2/1459) | categorial feature : True
Del Col : Exterior1st > trop de isnull (1/1459) | categorial feature : True
Del Col : Exterior2nd > trop de isnull (1/1459) | categorial feature : True
Del Col : MasVnrType > trop de isnull (16/1459) | categorial feature : True
Replace mean : MasVnrArea > trop de isnull (15/1459) | categorial feature : False
Del Col : BsmtQual > trop de isnull (44/1459) | categorial feature : True
Del Col : BsmtCond > trop de isnull (45/1459) | categorial feature : True
Del Col : BsmtExposure > trop de isnull (44/1459) | categorial feature : True
Del Col : BsmtFinType1 > trop de isnull (42/1459) | categorial feature : True
Replace mean : BsmtFinSF1 > trop de isnull (1/1459) | categorial feature : False
Del Co

In [227]:
df.shape, df_test.shape

((1460, 80), (1459, 57))

In [228]:
####### TRAIN pre pro #######
df = df.drop(columns = list_col_del) #del des mêmes colomnes qu'on a del dans df_test



X = df.drop(columns = [target])
list_columns = list(X.columns)



for column in list_columns:
    nb_isnull = df.isnull().sum()[column]
    if nb_isnull > 0:
        try:
            df[column].fillna((df[column].mean()), inplace=True)
            print(f'Replace mean : {column} > trop de isnull ({nb_isnull}/{len(df)}) | categorial feature : {is_string_dtype(df[column])}')
        except:
            print(f'ERROR replace mean DEL : {column} > trop de isnull ({nb_isnull}/{len(df)}) | categorial feature : {is_string_dtype(df[column])}')
            df = df.drop(column, 1)
            df_test = df_test.drop(column, 1)

Replace mean : MasVnrArea > trop de isnull (8/1460) | categorial feature : False
ERROR replace mean DEL : Electrical > trop de isnull (1/1460) | categorial feature : True
Replace mean : GarageYrBlt > trop de isnull (81/1460) | categorial feature : False


In [229]:
df.shape, df_test.shape

((1460, 56), (1459, 56))

In [230]:
#df = df[:-1] #on del la last ligne car c'est des val nules

In [231]:
#on supprime l'id temporairement pour tester sur ces valeures test
df_test_noid = df_test.drop(columns = ['Id']) #del des mêmes colomnes qu'on a del dans df_test



In [232]:
df.shape, df_test_noid.shape  #Le df normal fait 1 col de plus car il y a le sale price

((1460, 56), (1459, 55))

In [233]:
#X on prend toutes les colomnes sauf notre target
X = df.drop(columns = [target])

#Y : notre target qu'on passe en float pour le modèle
y = df[[target]].astype({target: float})


X_list_cat = list(X.select_dtypes(include=['object']).columns)
X_list_num = list(X.select_dtypes(include=['int64','float64']).columns)
print(f"Col catégorielles : {len(X_list_cat)}\nCol numériques : {len(X_list_num)}")

Col catégorielles : 20
Col numériques : 35


In [234]:
X.shape, df_test_noid.shape

((1460, 55), (1459, 55))

In [235]:
#On check si la transformation des col catégorielles est possible avec OneHotEncoder()
ok_list = []
errors_list = []

for col_cat in X_list_cat:
    try:
        df_bugtest = X[[col_cat]]
        df_bugtest = oneHot.fit_transform(df_bugtest)
        df_bugtest=pd.DataFrame.sparse.from_spmatrix(df_bugtest)
        ok_list.append(col_cat)
    except:
        errors_list.append(col_cat)
        #X_list_cat.remove(col_cat)
        
print(f"Cols cat ok : {len(ok_list)}\nCols cat errors : {len(errors_list)}\n\nDel cols cat name : {errors_list}")

Cols cat ok : 20
Cols cat errors : 0

Del cols cat name : []


In [236]:
#On check si la transformation des col numériques est possible avec RobustScaler()
ok_list = []
errors_list = []

for col_num in X_list_num:
    try:
        df_bugtest = X[[col_num]]

        scaler = RobustScaler()
        df_bugtest = scaler.fit_transform(df_bugtest)
        df_bugtest=pd.DataFrame(df_bugtest)
        ok_list.append(col_num)
    except:
        errors_list.append(col_num)
        X_list_num.remove(col_num)
print(f"Cols num ok : {len(ok_list)}\nCols num errors : {len(errors_list)}\n\nDel cols num name : {errors_list}")

Cols num ok : 35
Cols num errors : 0

Del cols num name : []


In [237]:
print("\n Var catégorielles")
for column_del in X_list_cat:


    #application des transformers 
    df_cat= df[X_list_cat]
    df_cat = df_cat.drop(column_del, 1)
    df_num = df[X_list_num]


    # Transformer num
    scaler = RobustScaler()
    df_num = scaler.fit_transform(df_num)
    df_num=pd.DataFrame(df_num)

    # Transformer cat
    oneHot = OneHotEncoder()
    df_cat = oneHot.fit_transform(df_cat)
    df_cat=pd.DataFrame.sparse.from_spmatrix(df_cat)

    #Concaténation des dataframes
    X=pd.concat([df_num,df_cat],axis=1)
    X.head()


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

    model = LinearRegression()
    result = model.fit(X_train, y_train)
    score = result.score(X_test, y_test)
    print(column_del, score)

    
print("\n Var numériques")
for column_del in X_list_num:


    #application des transformers 
    df_cat= df[X_list_cat]
    
    df_num = df[X_list_num]
    df_num = df_num.drop(column_del, 1)


    # Transformer num
    scaler = RobustScaler()
    df_num = scaler.fit_transform(df_num)
    df_num=pd.DataFrame(df_num)

    # Transformer cat
    oneHot = OneHotEncoder()
    df_cat = oneHot.fit_transform(df_cat)
    df_cat=pd.DataFrame.sparse.from_spmatrix(df_cat)

    #Concaténation des dataframes
    X=pd.concat([df_num,df_cat],axis=1)
    X.head()


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

    model = LinearRegression()
    result = model.fit(X_train, y_train)
    score = result.score(X_test, y_test)
    print(column_del, score)


 Var catégorielles
Street 0.8860853460713372
LotShape 0.8857448365932509
LandContour 0.8858474684531487
LotConfig 0.8851492254902538
LandSlope 0.88739755458049
Neighborhood 0.8798322442963211
Condition1 0.8854096492465112
Condition2 0.8910049969802096
BldgType 0.8851957074625495
HouseStyle 0.8895393776281293
RoofStyle 0.8839990784302537
RoofMatl 0.8965159858193698
ExterQual 0.8848800475256953
ExterCond 0.8849296700806332
Foundation 0.8862772121303205
Heating 0.8875711674663294
HeatingQC 0.8841380887071506
CentralAir 0.8859188119785002
PavedDrive 0.885754729214914
SaleCondition 0.887060675373301

 Var numériques
MSSubClass 0.8854193096568381
LotArea 0.8819119085399378
OverallQual 0.8715180934345462
OverallCond 0.8781341019826077
YearBuilt 0.8842072958623677
YearRemodAdd 0.8860364833754236
MasVnrArea 0.8816056657786067
BsmtFinSF1 0.8853450167937718
BsmtFinSF2 0.8853450167937789
BsmtUnfSF 0.8853450167937893
TotalBsmtSF 0.8852486836850973
1stFlrSF 0.8853450167938174
2ndFlrSF 0.88534501679

In [238]:
# On prend toutes les variables restantes pour générer le meilleur modèle


df_cat= df[X_list_cat]
df_num = df[X_list_num]
    
# Transformer num
scaler = RobustScaler()
df_num = scaler.fit_transform(df_num)
df_num=pd.DataFrame(df_num)

# Transformer cat
oneHot = OneHotEncoder()
df_cat = oneHot.fit_transform(df_cat)
df_cat=pd.DataFrame.sparse.from_spmatrix(df_cat)

#Concaténation des dataframes
X=pd.concat([df_num,df_cat],axis=1)

In [239]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [240]:
len(X_train),len(y_train),len(X_test),len(y_test)

(1387, 1387, 73, 73)

In [241]:
model = LinearRegression()
result = model.fit(X_train, y_train)
result.score(X_test,y_test)

0.8853450167937961

In [242]:
model = LinearRegression()
result = model.fit(X_train, y_train)
result.score(X_train,y_train)


0.911231437456793

In [243]:
result.predict(X_train)

array([[ 49711.6140257 ],
       [470316.9599372 ],
       [197787.15579765],
       ...,
       [121940.89149265],
       [168460.21163145],
       [195177.42246397]])

In [244]:
df_cat= df_test_noid[X_list_cat]
df_num = df_test_noid[X_list_num]
    
# Transformer num
scaler = RobustScaler()
df_num = scaler.fit_transform(df_num)
df_num=pd.DataFrame(df_num)

# Transformer cat
oneHot = OneHotEncoder()
df_cat = oneHot.fit_transform(df_cat)
df_cat=pd.DataFrame.sparse.from_spmatrix(df_cat)

#Concaténation des dataframes
df_test_noid=pd.concat([df_num,df_cat],axis=1)


#result.predict(df_test_noid)

In [245]:
df_test_noid.shape, X_train.shape


((1459, 149), (1387, 159))

In [246]:
#on concat les 2 df
df_test.shape, df.shape

((1459, 56), (1460, 56))

In [247]:
df_concat =pd.concat([df_test,df],axis=0)

In [248]:
df_concat.shape

(2919, 57)

In [249]:
df_concat.columns

Index(['Id', 'MSSubClass', 'LotArea', 'Street', 'LotShape', 'LandContour',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold',
       'SaleCondition', 'SalePrice'],
      dtype='object')

In [250]:
df_concat = df_concat.drop(columns=["Id"])

In [251]:
#X on prend toutes les colomnes sauf notre target
X = df_concat.drop(columns = [target])

#Y : notre target qu'on passe en float pour le modèle
y = df_concat[[target]].astype({target: float})


X_list_cat = list(X.select_dtypes(include=['object']).columns)
X_list_num = list(X.select_dtypes(include=['int64','float64']).columns)
print(f"Col catégorielles : {len(X_list_cat)}\nCol numériques : {len(X_list_num)}")

Col catégorielles : 20
Col numériques : 35


In [252]:
X.shape, y.shape

((2919, 55), (2919, 1))

In [253]:
# On prend toutes les variables restantes pour générer le meilleur modèle


df_cat= X[X_list_cat]
df_num = X[X_list_num]
    
# Transformer num
scaler = RobustScaler()
df_num = scaler.fit_transform(df_num)
df_num=pd.DataFrame(df_num)

# Transformer cat
oneHot = OneHotEncoder()
df_cat = oneHot.fit_transform(df_cat)
df_cat=pd.DataFrame.sparse.from_spmatrix(df_cat)

#Concaténation des dataframes
X=pd.concat([df_num,df_cat],axis=1)

In [254]:
X.shape

(2919, 159)

In [255]:
X_train = X[1459:]
X_test = X[:1459]

y_train = y[1459:]

X_train.shape,y_train.shape, X_test.shape

((1460, 159), (1460, 1), (1459, 159))

In [256]:
model = LinearRegression()
result = model.fit(X_train, y_train)
result.score(X_train,y_train)

0.9105397349577331

In [257]:
X_train_tempo, X_test_tempo, y_train_tempo, y_test_tempo = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [258]:
model = LinearRegression()
result = model.fit(X_train_tempo, y_train_tempo)
result.score(X_test_tempo,y_test_tempo)

0.8551952817105505

In [259]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(n_estimators=100)
result = model.fit(X_train_tempo, y_train_tempo)
result.score(X_test_tempo,y_test_tempo)
#KAGGLE SCORE : 0.16850

0.00684931506849315

In [260]:
model = RandomForestClassifier(n_estimators=100)
result = model.fit(X_train, y_train)
result.score(X_train,y_train)
#KAGGLE SCORE : 0.16850

0.9993150684931507

In [ ]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
result = clf.fit(X_train_tempo, y_train_tempo)
result.score(X_test_tempo,y_test_tempo)
#KAGGLE SCORE : 0.25699

In [ ]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
result = clf.fit(X_train, y_train)
result.score(X_train,y_train)
#KAGGLE SCORE : 0.25699

In [274]:
from sklearn.svm import LinearSVC
model = LinearSVC()
result = model.fit(X_train_tempo, y_train_tempo)
result.score(X_train_tempo,y_train_tempo)

0.6815068493150684

In [276]:
model = LinearRegression()
result = model.fit(X_train_tempo, y_train_tempo)
result.score(X_test_tempo,y_test_tempo)
#KAGGLE SCORE : 0.15631

0.8551952817105505

In [277]:
df_final = pd.DataFrame(columns = ['Id','SalePrice'])



for element_id, element_SalePrices in zip(list(df_test['Id']),list(result.predict(X_test))):
    print(element_id, element_SalePrices)
    df_final=df_final.append({'Id' : int(element_id) , 'SalePrice' : int(element_SalePrices)} , ignore_index=True)

1461 [118391.16205784]
1462 [166154.23092401]
1463 [181457.52580823]
1464 [194717.93506634]
1465 [230732.53845481]
1466 [172811.91788716]
1467 [183018.36482511]
1468 [155438.08770209]
1469 [192808.1944498]
1470 [118282.19660062]
1471 [173862.86670887]
1472 [100664.95850518]
1473 [97087.67931009]
1474 [146780.86369076]
1475 [114772.08044011]
1476 [390515.4635694]
1477 [282008.60669711]
1478 [290495.31010501]
1479 [284084.91493327]
1480 [477091.89282767]
1481 [312858.86375753]
1482 [215388.45884713]
1483 [175380.65402396]
1484 [163749.43809551]
1485 [184224.68405841]
1486 [190761.12439679]
1487 [321471.67918953]
1488 [241244.30078504]
1489 [214902.71465166]
1490 [246154.03370278]
1491 [199251.57769816]
1492 [84703.50239976]
1493 [166816.0625465]
1494 [293642.5813091]
1495 [291311.61736916]
1496 [218262.76367744]
1497 [173688.91471744]
1498 [148842.66167904]
1499 [148406.15481376]
1500 [142314.02446989]
1501 [174671.52929389]
1502 [131531.13750627]
1503 [290162.44091781]
1504 [242061.0468

1904 [128845.30219616]
1905 [202705.59316307]
1906 [171818.82409028]
1907 [248692.86611897]
1908 [102947.21932946]
1909 [121100.71589527]
1910 [113636.1391644]
1911 [232195.91860134]
1912 [288925.55082805]
1913 [157750.94669878]
1914 [69764.68455969]
1915 [333425.42683483]
1916 [54887.08594152]
1917 [238503.9481885]
1918 [151087.57781617]
1919 [174568.58615673]
1920 [164322.68602261]
1921 [367994.75846347]
1922 [331297.38284137]
1923 [239234.79357484]
1924 [239032.17335995]
1925 [217331.65513703]
1926 [370831.37791802]
1927 [137021.38258993]
1928 [171851.53861181]
1929 [129916.60126997]
1930 [126213.4155364]
1931 [148600.05691119]
1932 [135965.00587747]
1933 [199608.65227886]
1934 [183700.93803133]
1935 [166599.43948875]
1936 [185863.65228487]
1937 [181982.66820837]
1938 [163329.44095393]
1939 [298358.31247377]
1940 [221283.98330934]
1941 [165706.782277]
1942 [174217.72783877]
1943 [242023.4247819]
1944 [326061.03957174]
1945 [408441.18339433]
1946 [126306.00542671]
1947 [124590.930203

2268 [437452.90702526]
2269 [136891.73639975]
2270 [182049.93813997]
2271 [209002.88350406]
2272 [170853.33349443]
2273 [152594.1783821]
2274 [174224.21132024]
2275 [170385.02642438]
2276 [185069.38347311]
2277 [184752.1058019]
2278 [157330.97041317]
2279 [108183.56844]
2280 [107224.86182549]
2281 [142172.47625521]
2282 [167546.25059188]
2283 [115149.82686392]
2284 [102759.95046716]
2285 [149555.76944506]
2286 [120948.34910247]
2287 [384333.04313123]
2288 [293695.66732778]
2289 [363180.42325054]
2290 [442005.97132626]
2291 [336808.93652494]
2292 [419681.56204755]
2293 [459347.73861576]
2294 [345483.40176411]
2295 [477288.52492605]
2296 [311478.98525395]
2297 [372653.24650788]
2298 [368560.0144146]
2299 [342606.76986852]
2300 [332042.35391897]
2301 [306213.88680225]
2302 [279298.23183226]
2303 [267777.04851432]
2304 [285119.29109841]
2305 [233142.18120223]
2306 [218414.31178647]
2307 [213210.78452243]
2308 [226358.43586483]
2309 [313989.17197326]
2310 [231250.9976336]
2311 [228871.98403

2700 [137139.66627713]
2701 [131049.41762962]
2702 [88174.33459024]
2703 [95026.32263663]
2704 [153369.42513708]
2705 [92828.26892373]
2706 [80595.73031722]
2707 [102827.3339216]
2708 [134746.26846732]
2709 [112345.75391439]
2710 [120768.20685329]
2711 [309956.67216617]
2712 [370296.81389212]
2713 [167854.53438427]
2714 [136604.77610767]
2715 [160305.72988054]
2716 [128153.02722779]
2717 [202269.16787865]
2718 [218079.37397539]
2719 [148005.25434315]
2720 [180954.19130081]
2721 [121173.84923066]
2722 [162345.43967986]
2723 [135342.53042086]
2724 [109631.06182121]
2725 [126936.92336887]
2726 [159543.37544811]
2727 [192685.95503966]
2728 [182071.85683322]
2729 [143698.06147423]
2730 [154068.33524622]
2731 [132778.353285]
2732 [125573.18555095]
2733 [169604.89173388]
2734 [167230.75342944]
2735 [139192.11342208]
2736 [142786.1690362]
2737 [110081.42607461]
2738 [123983.41134703]
2739 [157080.52792826]
2740 [133616.93982252]
2741 [131876.02749416]
2742 [149246.88140826]
2743 [168810.431652

In [278]:
df_final = df_final[['Id','SalePrice']].astype(int)
df_final

,Id,SalePrice
0,1461,118391
1,1462,166154
2,1463,181457
3,1464,194717
4,1465,230732
...,...,...
1454,2915,80098
1455,2916,76125
1456,2917,176518
1457,2918,114252


In [279]:
df_final.to_csv("sample_submission3.csv", index=False)

In [270]:
len(df_final)

1459

In [103]:
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.model_selection import train_test_split


/home/adrienfontaine/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [104]:
# fit all models
clf = LazyClassifier(predictions=True)
models, predictions = clf.fit(X_train_tempo, X_test_tempo, y_train_tempo, y_test_tempo)

100%|██████████| 29/29 [09:45<00:00, 20.19s/it]


In [14]:
#  .8824513924709132

In [105]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.02,0.02,None,0.02,3.16
BernoulliNB,0.01,0.01,None,0.00,0.10
ExtraTreesClassifier,0.02,0.01,None,0.02,1.76
GaussianNB,0.01,0.01,None,0.00,0.19
AdaBoostClassifier,0.01,0.01,None,0.00,1.42
SVC,0.01,0.01,None,0.00,2.06
LogisticRegression,0.01,0.01,None,0.01,10.95
NearestCentroid,0.01,0.01,None,0.00,0.15
DummyClassifier,0.01,0.00,None,0.01,0.07
